<a href="https://colab.research.google.com/github/itohanosa/ito/blob/master/wCocoa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=A7-T99z218bTlQGD8s06xU3RiFzvaz48sGIhegQ9lS0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7Fu_g4KGkYcaulG5q3t8weiqR8uw0X-YIp3vHImcZBQZCTiDHpc8c

Successfully saved authorization token.


In [3]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [4]:
import folium
print(folium.__version__)

0.8.3


Define variables
This set of global variables will be used throughout. For this demo, you must have a Cloud Storage bucket into which you can write files. (learn more about creating Cloud Storage buckets). You'll also need to specify your Earth Engine username, i.e. users/USER_NAME on the Code Editor Assets tab.

In [6]:
# Your Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'abuitohanosa'

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'users/abuitohanosa/'

# Use Landsat 8 surface reflectance data for predictors.
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# Use these bands for prediction.
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This is a trianing/testing dataset of points with known land cover labels.
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')
# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landcover'
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 3

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'
# Export imagery in this region.
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])
# The name of the Earth Engine asset to be created by importing
# the classified image from the TFRecord file in Cloud Storage.
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/Classified_pixel_demo'

Prepare Landsat 8 imagery
First, make a cloud-masked median composite of Landsat 8 surface reflectance imagery from 2018. Check the composite by visualizing with folium.

In [30]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(BANDS).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = L8SR.filterDate('2020-01-01', '2020-12-31').map(maskL8sr).median()

#crop
roi = ee.FeatureCollection('users/abuitohanosa/gadm36_CIV_0')
image = image.clip(roi)

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[-5.4, 5.4])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [33]:
#td
LABEL_DATA = ee.FeatureCollection('users/abuitohanosa/TD/13467_a')

In [37]:
# Sample the image at the points and add a random column.
sample = image.sampleRegions(
  collection=LABEL_DATA, properties=[LABEL], scale=30).randomColumn()

# Partition the sample approximately 90-10.
training = sample.filter(ee.Filter.lt('random', 0.9))
testing = sample.filter(ee.Filter.gte('random', 0.9))

from pprint import pprint

# Print the first couple points to verify.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

{'training': {'geometry': None,
              'id': '00000000000000000001_0',
              'properties': {'B2': 0.031300000846385956,
                             'B3': 0.0685499981045723,
                             'B4': 0.054749999195337296,
                             'B5': 0.2919999957084656,
                             'B6': 0.17730000615119934,
                             'B7': 0.08824999630451202,
                             'random': 0.3192107094070944},
              'type': 'Feature'}}
{'testing': {'geometry': None,
             'id': '0000000000000000001f_0',
             'properties': {'B2': 0.03669999912381172,
                            'B3': 0.07429999858140945,
                            'B4': 0.04690000042319298,
                            'B5': 0.4004000127315521,
                            'B6': 0.2054000049829483,
                            'B7': 0.09314999729394913,
                            'random': 0.9966376096495262},
             'type': 'Feature

In [11]:
# Make sure you can see the output bucket.  You must have write access.
#print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET) 
 #   else 'Can not find output Cloud Storage bucket.')

In [60]:
# Create the tasks.
# Export an ee.FeatureCollection as an Earth Engine asset.
training_task = ee.batch.Export.table.toAsset(**{
    'collection': training,
    'description':'Training Export',
    'assetId': 'users/abuitohanosa/TD'
})
training_task.start()

testing_task = ee.batch.Export.table.toAsset(**{
    'collection': testing,
    'description':'Testing Export',
    'assetId': 'users/abuitohanosa/TD'
})
testing_task.start()


In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list())

Read into a tf.data.Dataset
Here we are going to read a file in Cloud Storage into a tf.data.Dataset. (these TensorFlow docs explain more about reading data into a Dataset). Check that you can read examples from the file. The purpose here is to ensure that we can read from the file without an error. The actual content is not necessarily human readable.